In [ ]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import sklearn.cluster as sk_cluster
import sklearn.feature_extraction.text as sk_text
import sklearn.metrics as metrics
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage      

%matplotlib inline
pd.set_option('display.max_colwidth', -1)

In [ ]:
cleaned_data = pd.read_csv('cleaned_data.csv')
clinton_trump_texts = pd.read_table('clinton_trump_user_classes.txt', names = ['UserID', 'Class'], sep='\t')

In [ ]:
clinton_trump_texts_agg = cleaned_data.groupby('UserID')['Hash'].sum()
clinton_trump_sklearn = pd.DataFrame({'User_id':clinton_trump_texts_agg.index,'All_hashtag':clinton_trump_texts_agg.values})

In [ ]:
lower_case = clinton_trump_sklearn.All_hashtag.str.lower()

In [ ]:
vectorizer = sk_text.TfidfVectorizer(stop_words='english',max_features=10, min_df=1, max_df=300)
matrix = vectorizer.fit_transform(lower_case)

In [ ]:
df = pd.DataFrame(matrix.todense(),columns =  vectorizer.get_feature_names())
df['UserID'] = clinton_trump_sklearn.User_id
df


In [ ]:
merged_df = pd.merge(df,clinton_trump_texts, on='UserID')
merged_df

## K-means

In [ ]:
%%time
kmeans = sk_cluster.KMeans(n_clusters=2, n_init=10, max_iter=300)
kmeans_label = kmeans.fit_predict(merged_df)

In [ ]:
print ('\nPrinting cluster assignment:')
print(kmeans_label)

In [ ]:
print('\n')
error = kmeans.inertia_       #SSE; Sum of squared distances of samples to their closest cluster center.
print ("The total error of the clustering is: ", error)

In [ ]:
true = np.array(merged_df.Class)
true

In [ ]:
confusion = metrics.confusion_matrix(true, kmeans.labels_, labels=[0, 1])   # first row: 0 ; second row: 1
print(confusion)

In [ ]:
metrics.precision_score(true, kmeans.labels_, average='weighted') # weighted: the average precision of all clusters is returned


In [ ]:
metrics.recall_score(true, kmeans.labels_, average='weighted')  # weighted: the average recall of all clusters is returned


In [ ]:
metrics.f1_score(true, kmeans.labels_, average= 'weighted')     # weighted: the average f1 of all clusters is returned


## Agglomerative Clustering

### MAX based

In [ ]:
%%time
ag1 = sk_cluster.AgglomerativeClustering(linkage = 'complete', n_clusters = 2)

In [ ]:
ag_labels1 = ag1.fit_predict(merged_df)

In [ ]:
print ('\nPrinting cluseter assignment:')
ag_labels1

In [ ]:
ag1 = linkage(merged_df, method='complete')

In [ ]:
dendrogram(ag1)  

In [ ]:
confusion = metrics.confusion_matrix(true, ag1.labels_, labels=[0, 1])   # first row: 0 ; second row: 1
print(confusion)

In [ ]:
metrics.precision_score(true, ag1.labels_, average='weighted') # weighted: the average precision of all clusters is returned


In [ ]:
metrics.recall_score(true, ag1.labels_, average='weighted')  # weighted: the average recall of all clusters is returned


In [ ]:
metrics.f1_score(true, ag1.labels_, average= 'weighted')     # weighted: the average f1 of all clusters is returned


### SSE based

In [ ]:
ag2 = sk_cluster.AgglomerativeClustering(linkage = 'ward', n_clusters = 2)   

In [ ]:
ag_labels2 = ag2.fit_predict(merged_df)

In [ ]:
print ('\nPrinting cluseter assignment:')
ag_labels2 

In [ ]:
ag2 = linkage(merged_df, method='ward') 

In [ ]:
dendrogram(ag2)  

In [ ]:
confusion = metrics.confusion_matrix(true, ag2.labels_, labels=[0, 1])   # first row: 0 ; second row: 1
print(confusion)

In [ ]:
metrics.precision_score(true, ag2.labels_, average='weighted') # weighted: the average precision of all clusters is returned


In [ ]:
metrics.recall_score(true, ag2.labels_, average='weighted')  # weighted: the average recall of all clusters is returned


In [ ]:
metrics.f1_score(true, ag2.labels_, average= 'weighted')     # weighted: the average f1 of all clusters is returned


## Task 1.2

### For k-means, look at the two centers (centroids) and print the top-30 hashtags/handles with the highest tfidf values.